### Gene deletions 

Main idea, that did not work as initally planed: 

The following script will be used to adjust the already created models further with gene deletions to further increase the similarity to the *e. coli* pathways, that are described in Huang, Zhongshi, et al paper. 
First we will see, if those knockouts change the growth rate and secondly we will set those genes to zero in our 3 different pathways. 



In [109]:
# This is just importing of the model and some tools to work with the GSM.
import numpy as np
from cobra.io import read_sbml_model
from cobra.util import create_stoichiometric_matrix
from cobra import Reaction, Metabolite
import pandas as pd
# Read model (central metabolism model of Escherichia coli)
model = read_sbml_model("iML1515.xml")
%run -i create_pathway.ipynb #runs script, has 

In [110]:
# creating the models with the respective pathways
nam_model = create_pathway(model.copy(),1)
na_model = create_pathway(model.copy(),2)
nr_model = create_pathway(model.copy(),3)
models = [(nam_model,'NAMPT'),(na_model,'NMN Synthetase'),(nr_model,'DeoD')]

In [111]:
import pandas
from time import time

from cobra.io import load_model
from cobra.flux_analysis import (
    single_gene_deletion, single_reaction_deletion, double_gene_deletion,
    double_reaction_deletion)

NAM Model: For the Nam Model it is not necessary to Knockout the purR gene, because it is not available in the GSM 

NA Model: For the NA Model it is necessary to knock out the pncC gene. The pncC is known as the gene: b2700 in the iML1515 model and is associated to the reaction NMNDA 

In [112]:
print('complete model: ', na_model.optimize())
with na_model:
    na_model.genes.b2700.knock_out()
    print('pncC knocked out: ', na_model.optimize())
    

complete model:  <Solution 0.877 at 0x7f7ff1cd71c0>
pncC knocked out:  <Solution 0.877 at 0x7f7ff1cd7190>


NR Model: For the NR Model it is important to knock out the nadR and ushA genes. The UshA gene is known as the b0480 gene in the iML1515 model and is associated to many different reactions. the nadR is known as the b4390 gene and associated to the NMNAT reaction.  

In [113]:
print('complete model: ', nr_model.optimize())
with nr_model:
    nr_model.genes.b0480.knock_out()
    print('ushA knocked out: ', nr_model.optimize())
    nr_model.genes.b4390.knock_out()
    print('nadR knocked out: ', nr_model.optimize())

complete model:  <Solution 0.877 at 0x7f7fd4588910>
ushA knocked out:  <Solution 0.877 at 0x7f7fd45888b0>
nadR knocked out:  <Solution 0.877 at 0x7f7fd4588850>


In the next step we would like to change our models according to the gene deletions. 

### NA Pathway: deleting gene b2700 (PncC) from the NA Model 

In [114]:
def remove_genes(
    model:"na_model",
    gene_list: Union[List["b2700"],Set["Gene"],List[str],Union[str]],
    remove_reactions: bool = True,
) -> None:
    
    rxn_set = set()
    for gene in model.genes.get_by_any(gene_list):
        gene.knock_out()
        rxn_set.update(gene.reactions)
    return [rxn for rxn in rxn_set if not rxn.functional]
    

### NAR Pathway: deleting gene b4390 (NadR)and the gene b0480 (UshA) from the NR Model 

In [115]:
from cobra.util import get_context
def delete_model_genes(
    model: "nr_model",
    gene_list: Union[List[ "b4390" ], Set["Gene"], List[str], Set[str]],
    cumulative_deletions: bool = True,
    disable_orphans: bool = 
    False,
) ->None:
    
    rxn_set = set()
    for gene in model.genes.get_by_any(gene_list):
        gene.knock_out()
        rxn_set.update(gene.reactions)
    return [rxn for rxn in rxn_set if not rxn.functional]

In [116]:
from cobra.util import get_context
def delete_model_genes(
    model: "nr_model",
    gene_list: Union[List[ "b0480" ], Set["Gene"], List[str], Set[str]],
    cumulative_deletions: bool = True,
    disable_orphans: bool = 
    False,
) ->None:
    
    rxn_set = set()
    for gene in model.genes.get_by_any(gene_list):
        gene.knock_out()
        rxn_set.update(gene.reactions)
    return [rxn for rxn in rxn_set if not rxn.functional]

In [117]:
nr_model.genes.b2700

Gene identifier,b2700
Name,pncC
Memory address,0x7f7fd878d820
Functional,True
In 1 reaction(s),NMNDA


In [118]:
na_model.genes. b2551

Gene identifier,b2551
Name,glyA
Memory address,0x7f7febdf2c70
Functional,True
In 6 reaction(s),"ALATA_D2, ALATA_L2, THFAT, THRA2, THRA, GHMT2r"


In [119]:
na_model.reactions.NMNDA

Reaction identifier,NMNDA
Name,Nicotinamide-nucleotide amidase
Memory address,0x7f7febb9bdf0
Stoichiometry,h2o_c + nmn_c --> nh4_c + nicrnt_c H2O H2O + NMN C11H14N2O8P --> Ammonium + Nicotinate D-ribonucleotide
GPR,b2700
Lower bound,0.0
Upper bound,1000.0


### Because the above described way didnt work, we are gonna set the bounds to 0 for the reaction NMNAD for all 3 models, which is linked to the gene b2700 (in the report) 

In the end we found out, that the genes b4390 (NadR)and b0480 (UshA), that were described in Huang, Zhongshi, et al paper. which convert NAM to NR are not associated to this distinct reaction in our GSM, which was proven with the BiGG database. 